In [1]:
import pandas as pd
import numpy as np
import random
from dateutil.relativedelta import relativedelta
from datetime import datetime, timedelta

# Prepare common functions-trade date generator, customer name generator etc

In [2]:
!pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 23.4 MB/s eta 0:00:00


In [3]:
from faker import Faker
faker = Faker()

In [4]:
faker_us = Faker('en_US')       # American
faker_uk = Faker('en_GB')       # British
faker_aus = Faker('en_AU')      # Australian
faker_ind = Faker('en_IN')      # Indian
Faker.seed(42)

In [6]:
# Set the path to the master data file and conditions file
ref_data_path = "RefData.xlsx"

# Load all sheets in both the files
ref_data = pd.read_excel(ref_data_path, sheet_name=None)

# Print the sheet names
print("Ref Data Sheets:", list(ref_data.keys()))

Ref Data Sheets: ['ProdType', 'ProdSubType', 'Currency', 'TD', 'BondType', 'BenchMark', 'Ratings', 'EqPrice', 'IndexLevel']


In [7]:
#Extract each ref data excel sheet into a separate dataframe
prodtype_master = ref_data["ProdType"]
prodsubtype_master = ref_data["ProdSubType"]
currency_master = ref_data["Currency"]
tradingdesk_master = ref_data["TD"]
bondtype_master = ref_data["BondType"]
benchmark_master = ref_data["BenchMark"]
ratings_master = ref_data["Ratings"]
equity_master = ref_data["EqPrice"]
index_master = ref_data["IndexLevel"]

In [8]:
# Strip leading/trailing spaces from all column names
currency_master["CurrencyCode"]=currency_master["CurrencyCode"].str.strip()
ratings_master.columns = ratings_master.columns.str.strip()
equity_master.columns = equity_master.columns.str.strip()
index_master.columns = index_master.columns.str.strip()

In [9]:
equity_master['Price'] = equity_master['Price'].str.replace('$','', regex=False)
equity_master['Price'] = pd.to_numeric(equity_master['Price'], errors='coerce')

In [10]:
#Generate trade dates - no deals on weekends
def generate_trade_date(category="auto"):
    """
    Generate a trade date ensuring:
    1. No weekends (Sat/Sun).
    2. Coverage guarantee: at least one deal per day in each category.
    """

    # Define ranges
    start_20 = datetime(2025, 11, 10)
    end_20   = datetime(2025, 12, 5)

    start_80 = datetime(2023, 1, 1)
    end_80   = datetime(2025, 11, 28)

    if category == "20":
        # All weekdays in Dec 1–5, 2025
        days_range = [(start_20 + timedelta(days=i))
                      for i in range((end_20 - start_20).days + 1)
                      if (start_20 + timedelta(days=i)).weekday() < 5]
        return random.choice(days_range)

    elif category == "80":
        # All weekdays in Jan 1, 2023 – Nov 28, 2025
        days_range = [(start_80 + timedelta(days=i))
                      for i in range((end_80 - start_80).days + 1)
                      if (start_80 + timedelta(days=i)).weekday() < 5]
        return random.choice(days_range)

    else:
        # Auto mode: 20% vs 80% split
        if random.random() < 0.2:
            return generate_trade_date("20")
        else:
            return generate_trade_date("80")


In [11]:
#Generate Customer and Counterparty data
#Weighted sampling across locales
def generate_mixed_name():
    locale_choice = random.choices(
        population=['us', 'uk', 'aus', 'ind'],
        weights=[0.3, 0.2, 0.2, 0.3],  # Adjust weights as needed
        k=1
    )[0]
    suffix_choice = ("Pub Ltd Company","Pvt Ltd Company","Banking Ltd",
                     "Non-Banking Fin Co", "Finance Company", "Mutual Fund",
                     "Brokers Ltd")[0]

    if locale_choice == 'us':
        name = faker_us.name()
    elif locale_choice == 'uk':
        name = faker_uk.name()
    elif locale_choice == 'aus':
        name = faker_aus.name()
    else:
        name = faker_ind.name()

    suffix_choice = random.choice([
        "Pub Ltd Company", "Pvt Ltd Company", "Banking Ltd",
        "Non-Banking Fin Co", "Finance Company", "Mutual Fund",
        "Brokers Ltd"
    ])
    name += " " + suffix_choice
    return name

In [12]:
#Define currency thresholds
currency_list = currency_master["CurrencyCode"].tolist()

explicit_weights = {
    "USD": 0.7,
    "EUR": 0.1,
    "GBP": 0.05,
    "CAD": 0.03,
    "INR": 0.02,
    "JPY": 0.02,
    "AUD": 0.02,
}

def choose_currency_weighted(currency_list, weights_dict):
    weights = [weights_dict.get(curr, 0.06 / (len(currency_list) - len(weights_dict)))
               if curr not in weights_dict else weights_dict[curr]
               for curr in currency_list]
    return random.choices(currency_list, weights=weights, k=1)[0]

In [13]:
exchange_rates = {
    "USD": 1.0,
    "EUR": 1.1,
    "GBP": 1.25,
    "SGD": 0.78,
    "INR": 0.012,
    "VEF": 0.08,
    "CAD": 0.73,
    "JPY": 0.0068,
    "ZAR": 0.056,
    "CNY": 0.14,
    "DKK": 0.15,
    "EGP": 0.021,
    "AUD": 0.65
}

In [14]:
ratings_list = ratings_master["Rating"].tolist()
rating_weights = {
    "AAA": 0.25,
    "AA": 0.20,
    "A": 0.20,
    "BBB": 0.15,
    "BB": 0.10,
    "B": 0.05,
    "CCC": 0.03,
    "CC": 0.01,
    "C": 0.007,
    "D": 0.003
}
weights = [rating_weights.get(r, 0.01) for r in ratings_list]


In [15]:
benchmark_list = benchmark_master["BenchMarkName"].tolist()
bm_weights = {
    "SOFR": 0.25,
    "SONIA": 0.20,
    "ESTR": 0.20,
    "MIBOR": 0.15,
    "EURIBOR": 0.10,
    }
weights_bm = [bm_weights.get(r, 0.01) for r in benchmark_list]


In [16]:
#Generate trade dates - no deals on weekends
def generate_trade_dateder(category="auto"):
    """
    Generate a trade date ensuring:
    1. No weekends (Sat/Sun).
    2. Coverage guarantee: at least one deal per day in each category.
    """

    # Define ranges
    start_80 = datetime(2025, 11, 1)
    end_80   = datetime(2025, 12, 5)

    start_20 = datetime(2023, 1, 1)
    end_20   = datetime(2025, 10, 31)

    if category == "80":
        # All weekdays in Nov 1 to Dec 5, 2025
        days_range = [(start_80 + timedelta(days=i))
                      for i in range((end_80 - start_80).days + 1)
                      if (start_80 + timedelta(days=i)).weekday() < 5]
        return random.choice(days_range)

    elif category == "20":
        # All weekdays in Jan 1, 2023 – Nov 1, 2025
        days_range = [(start_20 + timedelta(days=i))
                      for i in range((end_20 - start_20).days + 1)
                      if (start_20 + timedelta(days=i)).weekday() < 5]
        return random.choice(days_range)

    else:
        # Auto mode: 20% vs 80% split
        if random.random() < 0.2:
            return generate_trade_dateder("20")
        else:
            return generate_trade_dateder("80")

# Equity deals generation

In [17]:
#Generate equity deals
num_deals = random.randint(10000, 11000)
#num_deals = random.randint(10, 50)
portfolio_target_usd = 90_000_000_000   # 90 billion USD
eq_deals = []
deals_id_counter = 1

for i in range(num_deals):
    booking_systems_equity = random.choice(["Bacardi","Kondorplus","Bloomberg"])
    counter_party_type = random.choice(["Broker","CCP","OTC"])
    trading_desk = random.choice(tradingdesk_master['TradinDeskName'].tolist())
    order_type = random.choice(["Market","Limit","Stop"])
    port_strategy = random.choice(["Diversify","Rebalance","Active","Passive"])

    name = generate_mixed_name()
    # 1. Pick a random currency
    trade_currency = choose_currency_weighted(currency_list, explicit_weights)
    rate = exchange_rates[trade_currency]

    # 2. Generate gross_amount in USD terms (between 100k and 100m)
    gross_amount_usd = random.uniform(100_000, 100_000_000)

    # 3. Convert to local currency
    gross_amount_local = gross_amount_usd / rate

    # 4. Random price per unit (say between 10 and 500 in local currency)
    price = random.uniform(10, 500)

    # 5. Derive quantity = gross_amount_local / price
    trade_quantity = int(gross_amount_local / price)

    fee = random.choice([0.005,0.01])
    fees_commission = fee * gross_amount_local

    if counter_party_type in ["Broker", "CCP"]:
      execution_venue = random.choice(["Exchange", "MTF"])
    elif counter_party_type == "OTC":
      execution_venue = "OTC"

    trade_date = generate_trade_date()
    trade_date_str = trade_date.strftime("%d-%b-%Y")
    # Rule 1: Always Y if trade_date <= 10-Nov-2025
    cutoff_date = datetime(2025, 11, 10)

    if trade_date <= cutoff_date:
      confirmation_flg = "Y"
    else:
    # Rule 2: Randomly assign Y (80%) or N (20%)
      confirmation_flg = random.choices(["Y", "N"], weights=[0.9, 0.1], k=1)[0]

    cutoff_date = datetime(2025, 12, 5)
    settlement_date = trade_date + timedelta(days=2)
    if settlement_date <= cutoff_date:
    # 95% chance of Y, 5% chance of N
      settlement_status = random.choices(["Y", "N"], weights=[0.95, 0.05], k=1)[0]
    else:
    # Always N if after cutoff
      settlement_status = "N"

    settlement_stp = random.choices(["Y", "N"], weights=[0.98, 0.02], k=1)[0]

    trade_status = random.choices(["New", "Amended", "Cancelled"],weights=[0.95, 0.04, 0.01],k=1)[0]
    trade_cap_stp = random.choices(["Y", "N"], weights=[0.96, 0.04], k=1)[0]

    deals = {

        "Trade_ID": f"EQT{i+1:04}",
        "Booking_system": booking_systems_equity,
        "Trade_date": trade_date.strftime("%d-%b-%Y"),
        "Value_date":	trade_date.strftime("%d-%b-%Y"),
        "Confirmation Date":trade_date.strftime("%d-%b-%Y"),
        "Confirmation_flg": confirmation_flg,
        "Settlement_date":	settlement_date.strftime("%d-%b-%Y"),
        "Customer_ID": f"CUST{i+1:06}",
        "Customer Name": generate_mixed_name(),
        "Counterparty_Type":	counter_party_type,
        "Counterparty": generate_mixed_name(),
        "Trading_Desk_ID": trading_desk,
        "Product_type":	"Equity",
        "Product_subtype": 	"Cash Equity",
        "Buy_sell": random.choice(["Buy","Sell"]),
        "Trade_quantity":	trade_quantity,
        "Trade_currency":trade_currency,
        "Price":round(price, 2),
        "Gross_amount":	round(gross_amount_local, 2),
        "Gross_amount_USD": round(gross_amount_usd, 2),
        "Net_amount":	round(gross_amount_local,2),
        "Fees_commission":	round(fees_commission, 2),
        "Execution_venue": execution_venue,
        "Order_type":order_type,
        "Trade_status":	trade_status,
        "Settlement_status":	settlement_status,
        "Settlement_type":"Physical",
        "Trade_capture_stp": trade_cap_stp,
        "Settlement_stp": settlement_stp,
        "Legal_entity":"BKUS",
        "Portfolio_strategy":port_strategy

    }
    eq_deals.append(deals)


df_equity = pd.DataFrame(eq_deals)

In [ ]:
df_equity


,Trade_ID,Booking_system,Trade_date,Value_date,Confirmation Date,Confirmation_flg,Settlement_date,Customer_ID,Customer Name,Counterparty_Type,...,Gross_amount,Gross_amount_USD,Net_amount,Fees_commission,Execution_venue,Order_type,Trade_status,Settlement_status,Legal_entity,Portfolio_strategy
0,EQT0001,Bacardi,27-Sep-2023,27-Sep-2023,27-Sep-2023,Y,29-Sep-2023,CUST000001,Mohammed Gill-Ali Non-Banking Fin Co,Broker,...,2.311242e+07,25423660.50,2.311242e+07,115562.09,MTF,Stop,New,Y,BKUS,Rebalance
1,EQT0002,Bloomberg,05-Dec-2025,05-Dec-2025,05-Dec-2025,Y,07-Dec-2025,CUST000002,Upma Kapadia Banking Ltd,Broker,...,6.608231e+07,66082312.42,6.608231e+07,660823.12,MTF,Stop,New,N,BKUS,Rebalance
2,EQT0003,Kondorplus,23-Jan-2025,23-Jan-2025,23-Jan-2025,Y,25-Jan-2025,CUST000003,Corey Murphy Finance Company,Broker,...,5.442685e+07,68033562.38,5.442685e+07,272134.25,Exchange,Limit,New,Y,BKUS,Active
3,EQT0004,Kondorplus,28-Jul-2023,28-Jul-2023,28-Jul-2023,Y,30-Jul-2023,CUST000004,Advika Gokhale Brokers Ltd,OTC,...,5.413506e+09,64962068.24,5.413506e+09,54135056.87,OTC,Limit,New,Y,BKUS,Rebalance
4,EQT0005,Bacardi,02-Dec-2025,02-Dec-2025,02-Dec-2025,N,04-Dec-2025,CUST000005,Yasti D’Alia Pvt Ltd Company,OTC,...,8.106209e+07,63228432.16,8.106209e+07,810620.93,OTC,Market,New,Y,BKUS,Rebalance
5,EQT0006,Kondorplus,02-Dec-2025,02-Dec-2025,02-Dec-2025,Y,04-Dec-2025,CUST000006,Michelle Harmon Banking Ltd,CCP,...,6.248435e+07,62484347.88,6.248435e+07,624843.48,MTF,Market,New,N,BKUS,Passive
6,EQT0007,Bacardi,25-Aug-2023,25-Aug-2023,25-Aug-2023,Y,27-Aug-2023,CUST000007,Michelle Evans Mutual Fund,Broker,...,4.011082e+07,40110819.85,4.011082e+07,200554.10,MTF,Stop,New,Y,BKUS,Rebalance
7,EQT0008,Bacardi,03-Dec-2025,03-Dec-2025,03-Dec-2025,Y,05-Dec-2025,CUST000008,Owen Mitra Non-Banking Fin Co,Broker,...,1.827808e+07,18278078.60,1.827808e+07,182780.79,MTF,Stop,New,Y,BKUS,Rebalance
8,EQT0009,Bloomberg,05-Dec-2025,05-Dec-2025,05-Dec-2025,Y,07-Dec-2025,CUST000009,Udyati Sathe Pvt Ltd Company,CCP,...,4.579323e+07,45793225.45,4.579323e+07,457932.25,MTF,Limit,New,N,BKUS,Passive
9,EQT0010,Bacardi,18-Nov-2024,18-Nov-2024,18-Nov-2024,Y,20-Nov-2024,CUST000010,Bradley Robertson Banking Ltd,CCP,...,4.289496e+07,42894963.39,4.289496e+07,428949.63,Exchange,Stop,New,Y,BKUS,Active


In [18]:
df_equity.to_csv("df_equity.csv", index=False)


# Fixed Income (Bonds, NCDs, Secured Notes) data

In [19]:
from dateutil.relativedelta import relativedelta

def assign_issue_and_maturity(trade_date, product_subtype):
    """
    Given a trade_date (datetime) and product_subtype,
    return Issued_date and Maturity_date.
    """

    # Step 1: Issued_date = random date between 1 month and 1 year before trade_date
    months_back = random.randint(1, 12)
    issued_date = trade_date - relativedelta(months=months_back)

    # Step 2: Maturity_date based on product_subtype
    if product_subtype == "Bonds":
        years_forward = random.randint(8, 10)
    elif product_subtype == "NCD":
        years_forward = random.randint(5, 8)
    elif product_subtype == "Secured Notes":
        years_forward = random.randint(3, 5)
    else:
        years_forward = random.randint(5, 7)  # fallback

    # Safe addition of years
    maturity_date = issued_date + relativedelta(years=years_forward)

    return issued_date, maturity_date


In [20]:
def assign_price(face_value):
    """
    Set price relative to face value:
    - 80% chance: 5–30% higher than FV
    - 20% chance: 5–30% lower than FV
    """
    if random.random() < 0.8:
        # 80% case: premium
        multiplier = random.uniform(1.05, 1.30)
    else:
        # 20% case: discount
        multiplier = random.uniform(0.70, 0.95)  # 5–30% lower
    return round(face_value * multiplier, 2)


In [21]:
def assign_trade_quantity(product_subtype):
    if product_subtype == "Bonds":
        # Corporate bonds: 1,000 to 5,000,000 face value
        return random.randint(1000, 50000)
    elif product_subtype == "NCD":
        # NCDs: 10,000 to 100,000 (retail units)
        return random.randint(1000, 50000)
    elif product_subtype == "Secured Notes":
        # Secured notes: 100,000 to 1,000,000
        return random.randint(1000, 50000)
    else:
        return random.randint(1000, 1_000_000)  # fallback


In [22]:
from dateutil.relativedelta import relativedelta

def get_next_coupon_date(issued_date, trade_date, coupon_freq):
    # Step 1: set frequency in months
    freq_map = {
        "Annual": 12,
        "Semiannual": 6,
        "Quarterly": 3,
        "Monthly": 1
    }
    months = freq_map.get(coupon_freq, 6)  # default semiannual

    # Step 2: roll forward from issued_date until we pass trade_date
    coupon_date = issued_date
    while coupon_date <= trade_date:
        coupon_date += relativedelta(months=months)

    return coupon_date


In [23]:
#Generate bond, NCD and SN deals
num_deals = random.randint(5000, 5500)
portfolio_target_usd = 50_000_000_000   # 50 billion USD
fi_deals = []
deals_id_counter = 1

for i in range(num_deals):
    booking_systems_fixed = random.choice(["Murex","Riskone","Calypso","BloomTOMS"])
    counter_party_type = random.choice(["Broker","CCP","OTC"])
    trading_desk = random.choice(tradingdesk_master['TradinDeskName'].tolist())
    #order_type = random.choice(["Market","Limit","Stop"])
    port_strategy = random.choice(["Diversify","Rebalance","Active","Passive"])
    product_subtype = random.choices(["Bonds", "NCD","Secured Notes"], weights=[0.6, 0.2,0.2], k=1)[0]
    bond_rating = random.choices(ratings_list, weights=weights, k=1)[0]
    benchmark = random.choices(benchmark_list,weights=weights_bm,k=1)[0]
    coupon_freq = random.choice(["Monthly","Quarterly","Semiannual","Annual"])
    coupon_rate = round(random.uniform(3.0, 10.0), 1)
    face_value = random.choice([100, 1000, 10000])

    bond_seniority = random.choices(
    ["Senior Secured", "Senior Unsecured", "Subordinated", "Mezzanine", "Preferred"],
    weights=[0.4, 0.3, 0.2, 0.05, 0.05],k=1)[0]

    day_count = random.choices(
    ["30/360", "30E/360", "Actual/360", "Actual/365", "Actual/Actual (ISDA)", "Actual/Actual (ICMA)", "Business/252"],
    weights=[0.25, 0.15, 0.20, 0.15, 0.15, 0.08, 0.02], k=1)[0]

    coupon_type = random.choices(["Fixed Coupon","Floating Rate","Zero Coupon","Step-up Coupon"],
                                 weights =[0.6,0.3,0.2,0.1], k=1)[0]

    bond_type = random.choice(bondtype_master['BondType'].tolist())

    name = generate_mixed_name()
    # 1. Pick a random currency
    trade_currency = choose_currency_weighted(currency_list, explicit_weights)
    rate = exchange_rates[trade_currency]
    price = assign_price(face_value)
    trade_quantity = assign_trade_quantity(product_subtype)
    gross_amount_local = trade_quantity * price
    gross_amount_usd = gross_amount_local * rate
    gross_amount_local = round(gross_amount_local, 2)
    gross_amount_usd = round(gross_amount_usd, 2)

    fee = random.choice([0.0005,0.001])
    fees_commission = fee * gross_amount_local

    if counter_party_type in ["Broker", "CCP"]:
      execution_venue = random.choice(["Exchange", "MTF"])
    elif counter_party_type == "OTC":
      execution_venue = "OTC"

    trade_date = generate_trade_date()
    trade_date_str = trade_date.strftime("%d-%b-%Y")
    confirmation_date = trade_date + timedelta(days=1)
    # Rule 1: Always Y if trade_date <= 10-Nov-2025
    cutoff_date = datetime(2025, 11, 10)

    if trade_date <= cutoff_date:
      confirmation_flg = "Y"
    else:
    # Rule 2: Randomly assign Y (80%) or N (20%)
      confirmation_flg = random.choices(["Y", "N"], weights=[0.9, 0.1], k=1)[0]

    issued_date, maturity_date = assign_issue_and_maturity(trade_date, product_subtype)
    nxt_coupon = get_next_coupon_date(issued_date, trade_date, coupon_freq)
    settlement_date = nxt_coupon + timedelta(days=5)
    cutoff_date = datetime(2025, 12, 5)
    #settlement_date = trade_date + timedelta(days=5)
    if settlement_date <= cutoff_date:
    # 95% chance of Y, 5% chance of N
      settlement_status = random.choices(["Y", "N"], weights=[0.96, 0.04], k=1)[0]
    else:
    # Always N if after cutoff
      settlement_status = "N"

    bond_yield = (coupon_rate / price)
    accr_int = (face_value * coupon_rate)

    settlement_stp = random.choices(["Y", "N"], weights=[0.95, 0.05], k=1)[0]

    trade_status = random.choices(["New", "Amended", "Cancelled"],weights=[0.95, 0.04, 0.01],k=1)[0]
    trade_cap_stp = random.choices(["Y", "N"], weights=[0.82, 0.18], k=1)[0]

    deals = {

        "Trade_ID": f"FIC{i+1:04}",
        "Booking_system": booking_systems_fixed,
        "Trade_date": trade_date.strftime("%d-%b-%Y"),
        "Value_date":	trade_date.strftime("%d-%b-%Y"),
        "Confirmation Date":confirmation_date.strftime("%d-%b-%Y"),
        "Confirmation_flg": confirmation_flg,
        "Settlement_date":	settlement_date.strftime("%d-%b-%Y"),
        "Customer_ID": f"CUST{i+1:06}",
        "Customer Name": generate_mixed_name(),
        "Counterparty_Type":	counter_party_type,
        "Counterparty": generate_mixed_name(),
        "Trading_Desk_ID": trading_desk,
        "Product_type":	"FixedIncome",
        "Product_subtype": 	product_subtype,
        "Issuer_name": generate_mixed_name(),
        "Bond_type": bond_type,
        "Bond_seniority": bond_seniority,
        "Bond_rating": bond_rating,
        "FaceValue": face_value,
        "Price": price,
        "Clean_price": price,
        "coupon_type": coupon_type,
        "coupon_rate": coupon_rate,
        "coupon_freq":coupon_freq,
        "day_count": day_count,
        "Issue_date":issued_date,
        "Maturity_date": maturity_date,
        "Buy_sell": random.choice(["Buy","Sell"]),
        "Trade_quantity":	trade_quantity,
        "Trade_currency":trade_currency,
        "Gross_amount":	round(gross_amount_local, 2),
        "Gross_amount_USD": round(gross_amount_usd, 2),
        "Net_amount":	round(gross_amount_local,2),
        "Fees_commission":	round(fees_commission, 2),
        "Next_coupon_date":nxt_coupon,
        "Bond_yield":bond_yield,
        "Benchmark_rate": benchmark,
        "Accrued_int": accr_int,
        "Execution_venue": execution_venue,
        #"Order_type":order_type,
        "Trade_status":	trade_status,
        "Trade_capture_stp": trade_cap_stp,
        "Settlement_stp": settlement_stp,
        "Settlement_status":	settlement_status,
        "Settlement_type":"Physical",
        "Legal_entity":"BKUS",
        "Portfolio_strategy":port_strategy
    }
    fi_deals.append(deals)
df_fixedincome = pd.DataFrame(fi_deals)

In [24]:
df_fixedincome.to_csv("df_fixedincome.csv", index=False)

# Repos data

In [25]:
#Generate Repo and reverse repo deals
num_deals = random.randint(5000, 5500)
portfolio_target_usd = 148_000_000_000   # 148 billion USD
re_deals = []
deals_id_counter = 1

for i in range(num_deals):
    booking_systems_repo = random.choice(["Murex","FXGO","Calypso","BloomTOMS"])

    trading_desk = random.choice(tradingdesk_master['TradinDeskName'].tolist())
    port_strategy = random.choice(["Diversify","Rebalance","Active","Passive"])
    product_subtype = random.choices(["Repo", "ReverseRepo"], weights=[0.65, 0.35], k=1)[0]
    counter_party_type = random.choice(["Broker","OTC"])
    collat_type = random.choice(["Bonds-Soverign-CG","Bonds-Soverign-StateGovt",
                                 "Corporate Bonds", "MBS-ABS", "Equity"])
    name = generate_mixed_name()
    trade_currency = choose_currency_weighted(currency_list, explicit_weights)
    rate = exchange_rates[trade_currency]
    cash_amount_local = random.triangular(8000000, 1000000000, 10000000)
    cash_amount_usd = cash_amount_local * rate
    cash_amount_local = round(cash_amount_local, 2)
    cash_amount_usd = round(cash_amount_usd, 2)
    collat_value = cash_amount_local* 1.2
    collat_value = round(collat_value,2)

    tenor_days = random.choice([1, 7, 14, 30])
    repo_rate = round(random.uniform(3.0, 10.0), 1)
    ret_leg_amt_local = cash_amount_local * (1 + (repo_rate/100) * (tenor_days/360))
    ret_leg_amt_usd = ret_leg_amt_local * rate

    ret_leg_amt_local = round(ret_leg_amt_local,2)
    ret_leg_amt_usd  =  round(ret_leg_amt_usd,2)

    fee = random.choice([0.0005,0.001])
    fees_commission = fee * cash_amount_local

    execution_venue = 'OTC'


    trade_date = generate_trade_date()
    trade_date_str = trade_date.strftime("%d-%b-%Y")
    confirmation_date = trade_date + timedelta(days=1)
    # Rule 1: Always Y if trade_date <= 10-Nov-2025
    cutoff_date = datetime(2025, 11, 10)

    if trade_date <= cutoff_date:
      confirmation_flg = "Y"
    else:
    # Rule 2: Randomly assign Y (80%) or N (20%)
      confirmation_flg = random.choices(["Y", "N"], weights=[0.9, 0.1], k=1)[0]

    return_leg_date = (trade_date + timedelta(days=tenor_days))
    settlement_date = return_leg_date + timedelta(days=5)
    cutoff_date = datetime(2025, 12, 5)
    #settlement_date = trade_date + timedelta(days=5)
    if settlement_date <= cutoff_date:
    # 95% chance of Y, 5% chance of N
      settlement_status = random.choices(["Y", "N"], weights=[0.95, 0.05], k=1)[0]
    else:
    # Always N if after cutoff
      settlement_status = "N"
    settlement_stp = random.choices(["Y", "N"], weights=[0.95, 0.05], k=1)[0]
    trade_status = random.choices(["New", "Amended", "Cancelled"],weights=[0.95, 0.04, 0.01],k=1)[0]
    trade_cap_stp = random.choices(["Y", "N"], weights=[0.90, 0.1], k=1)[0]

    deals = {

        "Trade_ID": f"REPO{i+1:04}",
        "Booking_system": booking_systems_repo,
        "Trade_date": trade_date.strftime("%d-%b-%Y"),
        "Value_date":	trade_date.strftime("%d-%b-%Y"),
        "Confirmation Date":confirmation_date.strftime("%d-%b-%Y"),
        "Confirmation_flg": confirmation_flg,
        "Settlement_date":	settlement_date.strftime("%d-%b-%Y"),
        "Customer_ID": f"CUST{i+1:06}",
        "Customer Name": generate_mixed_name(),
        "Counterparty_Type":	counter_party_type,
        "Counterparty": generate_mixed_name(),
        "Trading_Desk_ID": trading_desk,
        "Product_type":	"FixedIncome",
        "Product_subtype": 	product_subtype,
        "Repo_type": product_subtype,
        "Repo_rate": repo_rate,
        "Repo_tenor":tenor_days,
        "Cash_leg_amt_local":cash_amount_local,
        "Cash_leg_amt_usd":cash_amount_usd,
        "Return_leg_date":return_leg_date.strftime("%d-%b-%Y"),
        "Return_leg_amt_local":ret_leg_amt_local,
        "Return_leg_amt_usd":ret_leg_amt_usd,
        "Collateral_type":collat_type,
        "Collateral_ID":f"COLL{i+1:04}",
        "Collateral_value":collat_value,
        "Trade_currency":trade_currency,
        "Fees_commission":	round(fees_commission, 2),
        "Execution_venue": execution_venue,
        "Trade_status":	trade_status,
        "Trade_capture_stp": trade_cap_stp,
        "Settlement_stp": settlement_stp,
        "Settlement_status":	settlement_status,
        "Settlement_type":"Physical",
        "Legal_entity":"BKUS",
        #"Portfolio_strategy":port_strategy
    }
    re_deals.append(deals)
df_repos= pd.DataFrame(re_deals)

In [26]:
df_repos.to_csv("df_repos.csv", index=False)

# Forex spot data

In [27]:
#Generate forex spot deals
num_deals = random.randint(8000, 8800)
portfolio_target_usd = 10_000_000_000   # 90 billion USD
fx_deals = []
deals_id_counter = 1

for i in range(num_deals):
    booking_systems_fx = random.choice(["Bacardi","Kondorplus","FXGO"])
    counter_party_type = random.choice(["Broker","OTC"])
    trading_desk = random.choice(tradingdesk_master['TradinDeskName'].tolist())
    execution_venue = random.choice(['Bloomberg','Reutuers','EBS'])
    #settlement_type = random.choice(["Cash","Physical"])

    name = generate_mixed_name()

    base_currency = random.choices(currency_list, weights=[explicit_weights.get(c, 1) for c in currency_list], k=1)[0]
    quote_currency = random.choice([c for c in currency_list if c != base_currency])
    currency_pair = f"{base_currency}/{quote_currency}"

    base_amount_usd = random.uniform(500_000, 10_000_000)
    rate_base_usd = exchange_rates[base_currency]
    base_amount = base_amount_usd / rate_base_usd

    raw_rate = exchange_rates[base_currency] / exchange_rates[quote_currency]
    shock = 1 + random.uniform(-0.15, 0.15)
    exchange_rate = raw_rate * shock
    quote_amount = base_amount * exchange_rate

    fee = random.choice([0.005,0.01])
    fees_commission = fee * base_amount

    trade_date = generate_trade_date()
    trade_date_str = trade_date.strftime("%d-%b-%Y")
    # Rule 1: Always Y if trade_date <= 10-Nov-2025
    cutoff_date = datetime(2025, 11, 10)

    if trade_date <= cutoff_date:
      confirmation_flg = "Y"
    else:
    # Rule 2: Randomly assign Y (80%) or N (20%)
      confirmation_flg = random.choices(["Y", "N"], weights=[0.9, 0.1], k=1)[0]

    cutoff_date = datetime(2025, 12, 5)
    settlement_date = trade_date + timedelta(days=2)
    if settlement_date <= cutoff_date:
    # 95% chance of Y, 5% chance of N
      settlement_status = random.choices(["Y", "N"], weights=[0.95, 0.05], k=1)[0]
    else:
    # Always N if after cutoff
      settlement_status = "N"
    settlement_stp = random.choices(["Y", "N"], weights=[0.98, 0.02], k=1)[0]

    trade_status = random.choices(["New", "Amended", "Cancelled"],weights=[0.95, 0.04, 0.01],k=1)[0]
    trade_cap_stp = random.choices(["Y", "N"], weights=[0.92, 0.18], k=1)[0]

    deals = {

        "Trade_ID": f"FXS{i+1:04}",
        "Booking_system": booking_systems_fx,
        "Trade_date": trade_date.strftime("%d-%b-%Y"),
        "Value_date":	trade_date.strftime("%d-%b-%Y"),
        "Confirmation Date":trade_date.strftime("%d-%b-%Y"),
        "Confirmation_flg": confirmation_flg,
        "Settlement_date":	settlement_date.strftime("%d-%b-%Y"),
        "Customer_ID": f"CUST{i+1:06}",
        "Customer Name": generate_mixed_name(),
        "Counterparty_Type":	counter_party_type,
        "Counterparty": generate_mixed_name(),
        "Trading_Desk_ID": trading_desk,
        "Product_type":	"Forex",
        "Product_subtype": 	"Forex-Spot",
        "Currency_pair":currency_pair,
        "Base_currency":base_currency ,
        "Quote_currency":quote_currency ,
        "Exchange_rate": round(exchange_rate,4),
        "Base_amount":	round(base_amount,2),
        "Quote_amount":  round(quote_amount,2),
        "Base_amount_usd": round(base_amount_usd,2),
        "Buy_sell": random.choice(["Buy","Sell"]),
        #"Trade_quantity":	trade_quantity,
        #"Price":round(price, 2),
        #"Net_amount":	round(gross_amount_local,2),
        "Fees_commission":	round(fees_commission, 2),
        "Execution_venue": execution_venue,
        #"Order_type":order_type,
        "Trade_status":	trade_status,
        "Trade_capture_stp": trade_cap_stp,
        "Settlement_stp": settlement_stp,
        "Settlement_status":	settlement_status,
        "Settlement_type":"Physical",
        "Legal_entity":"BKUS",
        #"Portfolio_strategy":port_strategy

    }
    fx_deals.append(deals)


df_fxspot = pd.DataFrame(fx_deals)

In [28]:
df_fxspot.to_csv("df_fxspot.csv", index=False)

# Derivatives-Forex data

In [29]:
#Generate forex derivatives data
num_deals = random.randint(5000, 5500)
portfolio_target_usd = 6000_000_000_000   # 90 billion USD
derfx_deals_list=[]

for i in range(num_deals):
    booking_systems_derfx = random.choice(['Murex','FrontArena','Summit','Sophis','FusionCap'])
    counter_party_type = random.choice(["Broker","OTC"])
    trading_desk = random.choice(tradingdesk_master['TradinDeskName'].tolist())
    port_strategy = random.choice(["Diversify","Rebalance","Active","Passive"])
    settlement_type = random.choice(["Cash","Physical"])
    option_style = random.choice(['European','American'])
    option_type = random.choice(["Call", "Put"])
    contract_code = random.choice(['EURUSD-H26-CME','GBPUSD-M25-ICE','JPYUSD-Z25-CME'])
    contract_size = random.choice([62500,100000,125000])
    num_contracts = random.randint(10,50)
    maturity_months  = random.choice([1, 3, 6, 12, 24, 36])
    product_subtype = random.choices(["FxSwaps", "FxForwards","FxOptions","FxFutures"],
                                        weights=[0.6, 0.27,0.1,0.03], k=1)[0]

    if product_subtype == "FxFutures":
          execution_venue = random.choice(["Exchange", "MTF"])
    else:
          execution_venue = "OTC"

    name = generate_mixed_name()
    base_currency = random.choices(currency_list, weights=[explicit_weights.get(c, 1) for c in currency_list], k=1)[0]
    quote_currency = random.choice([c for c in currency_list if c != base_currency])
    currency_pair = f"{base_currency}/{quote_currency}"

    base_amount_usd = random.uniform(10_000_000, 1_000_000_000)
    rate_base_usd = exchange_rates[base_currency]
    base_amount = base_amount_usd / rate_base_usd

    raw_rate = exchange_rates[base_currency] / exchange_rates[quote_currency]
    shock = 1 + random.uniform(-0.15, 0.15)
    shock_leg2 = 1 + random.uniform(-0.20, 0.20)
    exchange_rate = raw_rate * shock
    forward_rate = exchange_rate * shock_leg2
    futures_rate = exchange_rate  * shock_leg2
    swap_rate_leg1 = exchange_rate
    swap_rate_leg2 = exchange_rate * shock_leg2
    swap_point = (swap_rate_leg2-swap_rate_leg1)
    option_strike_rate = exchange_rate * shock_leg2
    quote_amount = base_amount * exchange_rate
    quote_amount2 = base_amount * exchange_rate * shock_leg2
    fee = random.choice([0.005,0.01])
    fees_commission = fee * base_amount
    premium_pct = random.uniform(0.01, 0.05)  # 1% to 5%
    premium_amount = base_amount * premium_pct
    margin_pct = random.uniform(0.02, 0.05)   # 2% to 5%
    initial_margin = round(base_amount * margin_pct, 2)

    trade_date = generate_trade_dateder()
    trade_date_str = trade_date.strftime("%d-%b-%Y")
    maturity_date = trade_date + relativedelta(months=maturity_months)
    # Confirmation flg - Rule 1: Always Y if trade_date <= 10-Nov-2025
    cutoff_date = datetime(2025, 11, 10)
    if trade_date <= cutoff_date:
        confirmation_flg = "Y"
    else:
    # Confirmation flg - Rule 2: Randomly assign Y (80%) or N (20%)
        confirmation_flg = random.choices(["Y", "N"], weights=[0.9, 0.1], k=1)[0]

    cutoff_date = datetime(2025, 12, 5)
    settlement_date = maturity_date + timedelta(days=2)
    if settlement_date <= cutoff_date:
    # 95% chance of Y, 5% chance of N
        settlement_status = random.choices(["Y", "N"], weights=[0.95, 0.05], k=1)[0]
    else:
        # Always N if after cutoff
        settlement_status = "N"
    settlement_stp = random.choices(["Y", "N"], weights=[0.92, 0.08], k=1)[0]

    trade_status = random.choices(["New", "Amended", "Cancelled"],weights=[0.95, 0.04, 0.01],k=1)[0]
    trade_cap_stp = random.choices(["Y", "N"], weights=[0.84, 0.16], k=1)[0]

#Common fields

    derfx_deals = {
      "Trade_ID": f"FXS{i+1:04}",
      "Booking_system": booking_systems_derfx,
      "Trade_date": trade_date.strftime("%d-%b-%Y"),
      "Value_date":	trade_date.strftime("%d-%b-%Y"),
      "Confirmation Date":trade_date.strftime("%d-%b-%Y"),
      "Confirmation_flg": confirmation_flg,
      #"Settlement_date":	settlement_date.strftime("%d-%b-%Y"),
      "Customer_ID": f"CUST{i+1:06}",
      "Customer Name": generate_mixed_name(),
      "Counterparty_Type":	counter_party_type,
      "Counterparty": generate_mixed_name(),
      "Trading_Desk_ID": trading_desk,
      "Product_type":	"Derivatives",
      "Product_subtype": 	product_subtype,
      "Currency_pair":currency_pair,
      "Base_currency":base_currency ,
      "Quote_currency":quote_currency ,
      "Exchange_rate": round(exchange_rate,4),
      #"Settlement_type": settlement_type,
      "Base_amount":	round(base_amount,2),
      "Quote_amount":  round(quote_amount,2),
      "Base_amount_usd": round(base_amount_usd,2),
      "Buy_sell": random.choice(["Buy","Sell"]),
      "Settlement_date": settlement_date.strftime("%d-%b-%Y"),
      "Fees_commission":	round(fees_commission, 2),
      "Execution_venue": execution_venue,
      "Trade_status":	trade_status,
      "Trade_capture_stp": trade_cap_stp,
      "Settlement_status":	settlement_status,
      "Legal_entity":"BKUS",
      "Portfolio_strategy":port_strategy
    }
      #Fx forward specific data
    if product_subtype =='FxForwards':
          derfx_deals.update(
              { "forward rate": round(forward_rate,4),
                "Maturity_date_fwd": maturity_date.strftime("%d-%b-%Y"),
                "Settlement_type": "Physical",
                "Settlement_stp":settlement_stp})

        #Fx future specific details
    elif product_subtype =='FxFutures':
          derfx_deals.update(
              {"Contract_code": contract_code,
              "Contract_size": contract_size,
              "futures_rate": round(futures_rate,4),
              "Maturity_date_fut": maturity_date.strftime("%d-%b-%Y"),
              "Number_of_contracts": num_contracts,
              "Initial Margin": initial_margin,
              "Settlement_type": "Physical",
              "Settlement_stp": settlement_stp
              }
          )


        #Fx swap specific fields
    elif product_subtype =='FxSwaps':
          derfx_deals.update(
              {"Near_leg_date": trade_date.strftime("%d-%b-%Y"),
                "Far_leg_date": maturity_date.strftime("%d-%b-%Y"),
                "Near_leg_rate": round(swap_rate_leg1,4),
                "Far_leg_rate": round(swap_rate_leg2,4),
                "Near_leg_amount": round(base_amount,2),
                "Far_leg_amount":round(quote_amount2,2),
                "Swap_points":round(swap_point,4),
                "Settlement_type":"Cash",
                "Settlement_stp": settlement_stp
              })


          #Fx option specific fields
    elif product_subtype =='FxOptions':
          derfx_deals.update({
                "Option_type":option_type,
                "Strike_price":round(option_strike_rate,4),
                "Premium": premium_amount,
                "Expiry_date":maturity_date.strftime("%d-%b-%Y"),
                "Style": option_style,
                "Settlement_type": settlement_type,
                "Settlement_stp":settlement_stp
              })


    derfx_deals_list.append(derfx_deals)


df_derfx = pd.DataFrame(derfx_deals_list)

In [30]:
df_derfx.to_csv('df_derfx.csv',index=False)

# Derivatives - Equity data

In [31]:
#Generate equity derivatives data
num_deals = random.randint(8000, 8800)
dereq_deals_list = []

for i in range(num_deals):
    booking_systems_dereq = random.choice(['Bacardi','FrontArena','Summit','Sophis','FusionCap'])
    counter_party_type = random.choice(["Broker","CCP","OTC"])
    trading_desk = random.choice(tradingdesk_master['TradinDeskName'].tolist())
    port_strategy = random.choice(["Diversify","Rebalance","Active","Passive"])
    settlement_type = random.choice(["Cash","Physical"])
    option_style = random.choice(['European','American'])
    option_type = random.choice(["Call", "Put"])
    num_contracts = random.randint(100,300)
    contract_size = random.choice([10,50,100,150,200,250])
    maturity_months  = random.choice([1, 3, 6, 12, 24, 36])
    notional_swp = random.triangular(8_000_000, 1_000_000_000, 10_000_000)
    float_cur_int_rate = round(random.uniform(3.0, 10.0), 1)
    shock_int = 1 + random.uniform(-0.15, 0.15)
    float_fut_int_rate = float_cur_int_rate * shock_int
    swap_point = (float_fut_int_rate - float_cur_int_rate)
    product_subtype = random.choices(
        ["EqSwaps", "EqFutures","EqOptions","IndFutures", "IndOptions"],
        weights=[0.01, 0.05, 0.10, 0.09, 0.75], k=1)[0]

    if product_subtype in ("IndOptions","IndFutures","EqFutures","EqOptions"):
        execution_venue = random.choice(["Exchange", "MTF","OTC"])
    else:
        execution_venue = "OTC"

    name = generate_mixed_name()
    contract_currency = "USD"

    row_eq = equity_master.sample(n=1).iloc[0]
    stock_name = row_eq['StockName']
    stock_cur_price = row_eq['Price']
    shock_fut = 1 + random.uniform(-0.15, 0.15)
    shock_opt = 1 + random.uniform(-0.2, 0.2)
    stock_fut_price = stock_cur_price * shock_fut
    stock_strike_price = stock_cur_price * shock_opt

    row_index = index_master.sample(n=1).iloc[0]
    index_name = row_index['IndexName']
    index_cur_level = row_index['Level']
    index_fut_level = index_cur_level * shock_fut
    index_strike_price = index_cur_level * shock_opt

    contract_amount_eq = stock_cur_price * num_contracts * contract_size
    contract_amount_index = index_cur_level * num_contracts * contract_size

    fee = random.choice([0.005, 0.01])
    premium_pct = random.uniform(0.01, 0.05)  # 1% to 5%
    premium_amount_eq = contract_amount_eq * premium_pct
    premium_amount_index = contract_amount_index * premium_pct
    margin_pct = random.uniform(0.02, 0.05)   # 2% to 5%
    initial_margin_eq = round(contract_amount_eq * margin_pct, 2)
    initial_margin_index = round(contract_amount_index * margin_pct, 2)

    trade_date = generate_trade_dateder()
    maturity_date = trade_date + relativedelta(months=maturity_months)

    # Confirmation flg - Rule 1: Always Y if trade_date <= 10-Nov-2025
    cutoff_confirm = datetime(2025, 11, 10)
    if trade_date <= cutoff_confirm:
        confirmation_flg = "Y"
    else:
        confirmation_flg = random.choices(["Y", "N"], weights=[0.9, 0.1], k=1)[0]

    # Settlement status rule using cutoff 05-Dec-2025
    settlement_cutoff = datetime(2025, 12, 5)
    settlement_date = maturity_date + timedelta(days=2)
    if settlement_date <= settlement_cutoff:
        settlement_status = random.choices(["Y", "N"], weights=[0.95, 0.05], k=1)[0]
    else:
        settlement_status = "N"

    settlement_stp = random.choices(["Y", "N"], weights=[0.95, 0.05], k=1)[0]
    trade_status = random.choices(["New", "Amended", "Cancelled"], weights=[0.95, 0.04, 0.01], k=1)[0]
    trade_cap_stp = random.choices(["Y", "N"], weights=[0.94, 0.16], k=1)[0]

    # base/common fields for this trade
    dereq_deals = {
        "Trade_ID": f"EQD{i+1:04}",
        "Booking_system": booking_systems_dereq,
        "Trade_date": trade_date.strftime("%d-%b-%Y"),
        "Value_date": trade_date.strftime("%d-%b-%Y"),
        "Confirmation Date": trade_date.strftime("%d-%b-%Y"),
        "Confirmation_flg": confirmation_flg,
        "Settlement_date": settlement_date.strftime("%d-%b-%Y"),
        "Customer_ID": f"CUST{i+1:06}",
        "Customer Name": generate_mixed_name(),
        "Counterparty_Type": counter_party_type,
        "Counterparty": generate_mixed_name(),
        "Trading_Desk_ID": trading_desk,
        "Product_type": "Derivatives",
        "Product_subtype": product_subtype,
        "Settlement_type": settlement_type,
        "Buy_sell": random.choice(["Buy","Sell"]),
        # fees_commission will be set per product below
        "Execution_venue": execution_venue,
        "Trade_status": trade_status,
        "Trade_capture_stp": trade_cap_stp,
        "Settlement_status": settlement_status,
        "Legal_entity": "BKUS",
        "Portfolio_strategy": port_strategy
    }

    # product-specific fields and set fees_commission & initial_margin accordingly
    if product_subtype == 'EqFutures':
        dereq_deals.update({
            "Stock_name": stock_name,
            "Stock_cur_price": stock_cur_price,
            "Contract_amount": contract_amount_eq,
            "Contract_code": f"EQDF{i+1:04}",
            "Contract_size": contract_size,
            "Stock_fut_price": stock_fut_price,
            "Maturity_date_fut": maturity_date.strftime("%d-%b-%Y"),
            "Number_of_contracts": num_contracts,
            "Initial Margin": initial_margin_eq,
            "Settlement_stp": settlement_stp
        })
        fees_commission = round(fee * contract_amount_eq, 2)

    elif product_subtype == 'EqSwaps':
        dereq_deals.update({
            "Stock_name": stock_name,
            "Stock_cur_price": stock_cur_price,
            "Notional": round(notional_swp, 2),
            "Floating_cur_int_rate": float_cur_int_rate,
            "Near_leg_date": trade_date.strftime("%d-%b-%Y"),
            "Far_leg_date": maturity_date.strftime("%d-%b-%Y"),
            "Stock_fut_price": stock_fut_price,
            "Floating_fut_int_rate": round(float_fut_int_rate, 4),
            "Swap_points": round(swap_point, 4),
            "Settlement_stp": settlement_stp
        })
        # use notional for fee calc
        fees_commission = round(fee * notional_swp, 2)

    elif product_subtype == 'EqOptions':
        dereq_deals.update({
            "Stock_name": stock_name,
            "Stock_cur_price": stock_cur_price,
            "Contract_size": contract_size,
            "Number_of_contracts": num_contracts,
            "Contract_amount": contract_amount_eq,
            "Option_type": option_type,
            "Strike_price": round(stock_strike_price, 4),
            "Premium": round(premium_amount_eq, 2),
            "Expiry_date": maturity_date.strftime("%d-%b-%Y"),
            "Style": option_style,
            "Initial Margin": initial_margin_eq,
            "Settlement_stp": settlement_stp
        })
        fees_commission = round(fee * contract_amount_eq, 2)

    elif product_subtype == 'IndFutures':
        dereq_deals.update({
            "Index_name": index_name,
            "Index_cur_level": index_cur_level,
            "Contract_amount": contract_amount_index,
            "Contract_code": f"INDF{i+1:04}",
            "Contract_size": contract_size,
            "Index_fut_level": index_fut_level,
            "Maturity_date_fut": maturity_date.strftime("%d-%b-%Y"),
            "Number_of_contracts": num_contracts,
            "Initial Margin": initial_margin_index,
            "Settlement_stp": settlement_stp
        })
        fees_commission = round(fee * contract_amount_index, 2)

    elif product_subtype == 'IndOptions':
        dereq_deals.update({
            "Index_name": index_name,
            "Index_cur_level": index_cur_level,
            "Contract_size": contract_size,
            "Number_of_contracts": num_contracts,
            "Contract_amount": contract_amount_index,
            "Option_type": option_type,
            "Strike_price": round(index_strike_price, 4),
            "Premium": round(premium_amount_index, 2),
            "Expiry_date": maturity_date.strftime("%d-%b-%Y"),
            "Style": option_style,
            "Initial Margin": initial_margin_index,
            "Settlement_stp": settlement_stp
        })
        fees_commission = round(fee * contract_amount_index, 2)

    else:
        fees_commission = 0.0

    dereq_deals["Fees_commission"] = fees_commission

    dereq_deals_list.append(dereq_deals)

df_dereq = pd.DataFrame(dereq_deals_list)

# Show a quick preview
print(f"Generated {len(df_dereq)} deals. Columns: {list(df_dereq.columns)}")
print(df_dereq.head().to_string(index=False))

Generated 8016 deals. Columns: ['Trade_ID', 'Booking_system', 'Trade_date', 'Value_date', 'Confirmation Date', 'Confirmation_flg', 'Settlement_date', 'Customer_ID', 'Customer Name', 'Counterparty_Type', 'Counterparty', 'Trading_Desk_ID', 'Product_type', 'Product_subtype', 'Settlement_type', 'Buy_sell', 'Execution_venue', 'Trade_status', 'Trade_capture_stp', 'Settlement_status', 'Legal_entity', 'Portfolio_strategy', 'Index_name', 'Index_cur_level', 'Contract_size', 'Number_of_contracts', 'Contract_amount', 'Option_type', 'Strike_price', 'Premium', 'Expiry_date', 'Style', 'Initial Margin', 'Settlement_stp', 'Fees_commission', 'Stock_name', 'Stock_cur_price', 'Contract_code', 'Index_fut_level', 'Maturity_date_fut', 'Stock_fut_price', 'Notional', 'Floating_cur_int_rate', 'Near_leg_date', 'Far_leg_date', 'Floating_fut_int_rate', 'Swap_points']
Trade_ID Booking_system  Trade_date  Value_date Confirmation Date Confirmation_flg Settlement_date Customer_ID                    Customer Name Count

In [32]:
df_dereq.to_csv('df_dereq.csv',index=False)

# Derivatives - Interest rate

In [33]:
# Generate interest rate derivatives data
num_deals = random.randint(5000, 5500)
derint_deals_list = []

for i in range(num_deals):
    booking_systems_derint = random.choice(['Riskone','Kondorplus','Summit','Sophis','FusionCap'])
    counter_party_type = random.choice(["Broker","OTC"])
    trading_desk = random.choice(tradingdesk_master['TradinDeskName'].tolist())
    port_strategy = random.choice(["Diversify","Rebalance","Active","Passive"])
    benchmark = random.choices(benchmark_list,weights=weights_bm,k=1)[0]
    settlement_type = random.choice(["Cash","Physical"])
    option_style = random.choice(['European','American'])
    option_type = random.choice(["Call", "Put"])
    maturity_months  = random.choice([6, 12, 24, 36])
    effective_months  = random.choice([1,2, 3])
    float_cur_int_rate = round(random.uniform(3.0, 10.0), 1)
    shock_int = 1 + random.uniform(-0.15, 0.15)
    float_fut_int_rate = float_cur_int_rate * shock_int
    product_subtype = random.choices(
        ["IntRateSwaps", "IntRateOptions","IntRateFRA"],
        weights=[0.8, 0.03, 0.17], k=1)[0]

    execution_venue = "OTC"

    name = generate_mixed_name()
    contract_currency = "USD"

    contract_amount = random.triangular(8000000, 1000000000, 10000000)
    contract_amount = round(contract_amount,2)

    int_rate_cur_leg1 = round(random.uniform(3.0, 10.0), 1)
    int_rate_cur_leg2 = round(random.uniform(4.0, 9.0), 1)
    shock_leg1 = 1 + random.uniform(-0.15, 0.15)
    shock_leg2 = 1 + random.uniform(-0.15, 0.15)
    int_rate_fut_leg1 = int_rate_cur_leg1 * shock_leg1
    int_rate_fut_leg2 = int_rate_cur_leg2 * shock_leg2

    int_rate_option = round(random.uniform(5.0, 12.0), 1)
    strike_price = int_rate_option *shock_leg1

    int_rate_fra = round(random.uniform(2.0, 12.0), 1)

    fee = random.choice([0.005, 0.01])
    premium_pct = random.uniform(0.01, 0.05)  # 1% to 5%
    premium_amount = contract_amount * premium_pct

    margin_pct = random.uniform(0.02, 0.05)   # 2% to 5%
    initial_margin = round(contract_amount * margin_pct, 2)
    #

    trade_date = generate_trade_dateder()
    maturity_date = trade_date + relativedelta(months=maturity_months)
    effective_date = trade_date + relativedelta(months=effective_months)

    # Confirmation flg - Rule 1: Always Y if trade_date <= 10-Nov-2025
    cutoff_confirm = datetime(2025, 11, 10)
    if trade_date <= cutoff_confirm:
        confirmation_flg = "Y"
    else:
        confirmation_flg = random.choices(["Y", "N"], weights=[0.9, 0.1], k=1)[0]

    # Settlement status rule using cutoff 05-Dec-2025
    settlement_cutoff = datetime(2025, 12, 5)
    settlement_date = maturity_date + timedelta(days=2)
    if settlement_date <= settlement_cutoff:
        settlement_status = random.choices(["Y", "N"], weights=[0.95, 0.05], k=1)[0]
    else:
        settlement_status = "N"

    settlement_stp = random.choices(["Y", "N"], weights=[0.92, 0.08], k=1)[0]
    trade_status = random.choices(["New", "Amended", "Cancelled"], weights=[0.95, 0.04, 0.01], k=1)[0]
    trade_cap_stp = random.choices(["Y", "N"], weights=[0.90, 0.1], k=1)[0]

    # base/common fields for this trade
    derint_deals = {
        "Trade_ID": f"INTD{i+1:04}",
        "Trade_currency": contract_currency,
        "Booking_system": booking_systems_derint,
        "Trade_date": trade_date.strftime("%d-%b-%Y"),
        "Value_date": trade_date.strftime("%d-%b-%Y"),
        "Confirmation Date": trade_date.strftime("%d-%b-%Y"),
        "Confirmation_flg": confirmation_flg,
        "Settlement_date": settlement_date.strftime("%d-%b-%Y"),
        "Customer_ID": f"CUST{i+1:06}",
        "Customer Name": generate_mixed_name(),
        "Counterparty_Type": counter_party_type,
        "Counterparty": generate_mixed_name(),
        "Trading_Desk_ID": trading_desk,
        "Product_type": "Derivatives",
        "Product_subtype": product_subtype,
        #"Settlement_type": settlement_type,
        "Buy_sell": random.choice(["Buy","Sell"]),
        # fees_commission will be set per product below
        "Execution_venue": execution_venue,
        "Trade_status": trade_status,
        "Trade_capture_stp": trade_cap_stp,
        "Settlement_status": settlement_status,
        "Legal_entity": "BKUS",
        "Portfolio_strategy": port_strategy
    }

    # product-specific fields and set fees_commission & initial_margin accordingly

    if product_subtype == 'IntRateSwaps':
        derint_deals.update({
            "Notional": contract_amount,
            "Benchmark_rate": benchmark,
            "Int_rate_cur_leg1":int_rate_cur_leg1,
            "Int_rate_cur_leg2": int_rate_cur_leg2,
            "Near_leg_date": trade_date.strftime("%d-%b-%Y"),
            "Far_leg_date": maturity_date.strftime("%d-%b-%Y"),
            "Int_rate_fut_leg1":round(int_rate_fut_leg1,1),
            "Int_rate_fut_leg2": round(int_rate_fut_leg2,1),
            "Settlement_type": "Cash",
            "Settlement_stp": settlement_stp
        })
        # use notional for fee calc
        fees_commission = round(fee * contract_amount, 2)

    elif product_subtype == 'IntRateOptions':
        derint_deals.update({
            "Notional": contract_amount,
            "Benchmark_rate": benchmark,
            "Interest Rate": int_rate_option,
            "Option_type": option_type,
            "Strike_price": round(strike_price,2),
            "Premium": round(premium_amount, 2),
            "Expiry_date": maturity_date.strftime("%d-%b-%Y"),
            "Style": option_style,
            "Settlement_type": settlement_type,
            "Initial Margin": initial_margin,
            "Settlement_stp": settlement_stp
        })
        fees_commission = round(fee * contract_amount, 2)

    elif product_subtype == 'IntRateFRA':
        derint_deals.update({
            "Notional": contract_amount,
            "Benchmark_rate": benchmark,
            "Interest Rate": int_rate_fra,
            "Effective_date": effective_date,
            "Maturity_date_fra": maturity_date.strftime("%d-%b-%Y"),
            "Settlement_type": "Cash",
            "Settlement_stp": settlement_stp

        })
        fees_commission = round(fee * contract_amount, 2)

    else:
        fees_commission = 0.0

    derint_deals["Fees_commission"] = fees_commission

    derint_deals_list.append(derint_deals)

df_derint = pd.DataFrame(derint_deals_list)

# Show a quick preview
print(f"Generated {len(df_derint)} deals. Columns: {list(df_derint.columns)}")
print(df_derint.head().to_string(index=False))

Generated 5267 deals. Columns: ['Trade_ID', 'Trade_currency', 'Booking_system', 'Trade_date', 'Value_date', 'Confirmation Date', 'Confirmation_flg', 'Settlement_date', 'Customer_ID', 'Customer Name', 'Counterparty_Type', 'Counterparty', 'Trading_Desk_ID', 'Product_type', 'Product_subtype', 'Buy_sell', 'Execution_venue', 'Trade_status', 'Trade_capture_stp', 'Settlement_status', 'Legal_entity', 'Portfolio_strategy', 'Notional', 'Benchmark_rate', 'Int_rate_cur_leg1', 'Int_rate_cur_leg2', 'Near_leg_date', 'Far_leg_date', 'Int_rate_fut_leg1', 'Int_rate_fut_leg2', 'Settlement_type', 'Settlement_stp', 'Fees_commission', 'Interest Rate', 'Option_type', 'Strike_price', 'Premium', 'Expiry_date', 'Style', 'Initial Margin', 'Effective_date', 'Maturity_date_fra']
Trade_ID Trade_currency Booking_system  Trade_date  Value_date Confirmation Date Confirmation_flg Settlement_date Customer_ID                Customer Name Counterparty_Type                Counterparty Trading_Desk_ID Product_type Product_s

In [34]:
df_derint.to_csv('df_derint.csv',index=False)

# Derivatives - Credit

In [35]:
# Generate credit derivatives data
num_deals = random.randint(1000, 1100)
dercr_deals_list = []

for i in range(num_deals):
    booking_systems_dercr = random.choice(['Riskone','Kondorplus','Summit','Sophis','FusionCap'])
    counter_party_type = random.choice(["Broker","OTC"])
    trading_desk = random.choice(tradingdesk_master['TradinDeskName'].tolist())
    port_strategy = random.choice(["Diversify","Rebalance","Active","Passive"])
    benchmark = random.choices(benchmark_list,weights=weights_bm,k=1)[0]
    settlement_type = random.choice(["Cash","Physical"])
    maturity_months  = random.choice([6, 12, 24, 36])
    effective_months  = random.choice([1,2, 3])
    product_subtype = random.choices(
        ["CDS", "TRS"],
        weights=[0.8, 0.2], k=1)[0]

    execution_venue = "OTC"

    name = generate_mixed_name()
    contract_currency = "USD"

    asset_type = random.choice(["Bond","NCD","SecuredNote","Equity"])

    contract_amount = random.triangular(25000000, 1000000000, 100000000)
    contract_amount = round(contract_amount,2)


    cds_spread = round(random.uniform(3.0, 10.0), 1)
    trs_spread = round(random.uniform(4.0, 9.0), 1)
    shock_cds = 1 + random.uniform(-0.15, 0.15)
    shock_trs = 1 + random.uniform(-0.15, 0.15)
    cds_spread = cds_spread * shock_cds
    trs_spread = trs_spread* shock_trs

    recovery_pct = random.uniform(0.3, 0.5)
    recovery_rate = f"{round(recovery_pct * 100)}%"

    fee = random.choice([0.005, 0.01])
    premium_pct = random.uniform(0.01, 0.05)  # 1% to 5%
    premium_amount = contract_amount * premium_pct
    margin_pct = random.uniform(0.02, 0.05)   # 2% to 5%
    variation_margin = round(contract_amount * margin_pct, 2)


    trade_date = generate_trade_dateder()
    maturity_date = trade_date + relativedelta(months=maturity_months)

    # Confirmation flg - Rule 1: Always Y if trade_date <= 10-Nov-2025
    cutoff_confirm = datetime(2025, 11, 10)
    if trade_date <= cutoff_confirm:
        confirmation_flg = "Y"
    else:
        confirmation_flg = random.choices(["Y", "N"], weights=[0.9, 0.1], k=1)[0]

    # Settlement status rule using cutoff 05-Dec-2025
    settlement_cutoff = datetime(2025, 12, 5)
    settlement_date = maturity_date + timedelta(days=2)
    if settlement_date <= settlement_cutoff:
        settlement_status = random.choices(["Y", "N"], weights=[0.95, 0.05], k=1)[0]
    else:
        settlement_status = "N"

    settlement_stp = random.choices(["Y", "N"], weights=[0.92, 0.08], k=1)[0]
    trade_status = random.choices(["New", "Amended", "Cancelled"], weights=[0.95, 0.04, 0.01], k=1)[0]
    trade_cap_stp = random.choices(["Y", "N"], weights=[0.85, 0.15], k=1)[0]

    # base/common fields for this trade
    dercr_deals = {
        "Trade_ID": f"CRD{i+1:04}",
        "Trade_currency": contract_currency,
        "Booking_system": booking_systems_dercr,
        "Trade_date": trade_date.strftime("%d-%b-%Y"),
        "Value_date": trade_date.strftime("%d-%b-%Y"),
        "Confirmation Date": trade_date.strftime("%d-%b-%Y"),
        "Confirmation_flg": confirmation_flg,
        "Settlement_date": settlement_date.strftime("%d-%b-%Y"),
        "CP_ID": f"CP{i+1:06}",
        "Counterparty_Type": counter_party_type,
        "Counterparty": generate_mixed_name(),
        "Trading_Desk_ID": trading_desk,
        "Product_type": "Derivatives",
        "Product_subtype": product_subtype,
        "Settlement_type": settlement_type,
        "Buy_sell": random.choice(["Buy","Sell"]),
        "Execution_venue": execution_venue,
        "Trade_status": trade_status,
        "Trade_capture_stp": trade_cap_stp,
        "Settlement_status": settlement_status,
        "Legal_entity": "BKUS",
        "Portfolio_strategy": port_strategy
    }

    if product_subtype == 'CDS':
        dercr_deals.update({
            "Notional": contract_amount,
            "CDS_spread":cds_spread,
            "CDS_Tenor": maturity_months,
            "Recovery_rate": recovery_rate,
            "Variation_Margin": variation_margin,
            "Settlement_stp": settlement_stp
        })
        # use notional for fee calc
        fees_commission = round(fee * contract_amount, 2)

    elif product_subtype == 'TRS':
        dercr_deals.update({
            "Asset_type": asset_type,
            "Asset_ID": f"ASS{i+1:04}",
            "Notional": contract_amount,
            "Benchmark_rate": benchmark,
            "TRS_spread": round(trs_spread,2),
            "Reset_freq": effective_months,
            "Variation_Margin": variation_margin,
            "Settlement_stp": settlement_stp
        })
        fees_commission = round(fee * contract_amount, 2)

    else:
        fees_commission = 0.0

    dercr_deals["Fees_commission"] = fees_commission

    dercr_deals_list.append(dercr_deals)

df_dercr = pd.DataFrame(dercr_deals_list)

# Show a quick preview
print(f"Generated {len(df_dercr)} deals. Columns: {list(df_dercr.columns)}")
print(df_dercr.head().to_string(index=False))

Generated 1025 deals. Columns: ['Trade_ID', 'Trade_currency', 'Booking_system', 'Trade_date', 'Value_date', 'Confirmation Date', 'Confirmation_flg', 'Settlement_date', 'CP_ID', 'Counterparty_Type', 'Counterparty', 'Trading_Desk_ID', 'Product_type', 'Product_subtype', 'Settlement_type', 'Buy_sell', 'Execution_venue', 'Trade_status', 'Trade_capture_stp', 'Settlement_status', 'Legal_entity', 'Portfolio_strategy', 'Notional', 'CDS_spread', 'CDS_Tenor', 'Recovery_rate', 'Variation_Margin', 'Settlement_stp', 'Fees_commission', 'Asset_type', 'Asset_ID', 'Benchmark_rate', 'TRS_spread', 'Reset_freq']
Trade_ID Trade_currency Booking_system  Trade_date  Value_date Confirmation Date Confirmation_flg Settlement_date    CP_ID Counterparty_Type                 Counterparty Trading_Desk_ID Product_type Product_subtype Settlement_type Buy_sell Execution_venue Trade_status Trade_capture_stp Settlement_status Legal_entity Portfolio_strategy     Notional  CDS_spread  CDS_Tenor Recovery_rate  Variation_Mar

In [36]:
df_dercr.to_csv('df_dercr.csv',index=False)

# Collateral margin call data

In [37]:
#Generate margin calls data
import random
num_calls = random.randint(3000, 3300)
portfolio_target_usd = 90_000_000_000   # 90 billion USD
margin_calls = []
deals_id_counter = 1

for i in range(num_calls):
    call_date = generate_trade_dateder()
    call_source = random.choices(['Acadia','Manual'], weights = [0.95,0.05],k=1)[0]
    mar_type = random.choices(['InitialMargin','VariationMargin','NetCashMargin'],weights = [0.35,0.60,0.05],k=1)[0]
    call_dir = random.choice(["CallReceived","CallMade"])
    contract_amount = random.triangular(8000000, 1000000000, 10000000)
    margin_pct = random.uniform(0.02, 0.05)   # 2% to 5%
    mar_amount = round(contract_amount * margin_pct, 2)
    call_result = random.choices(["Agreed","Challenged","Disputed","Pending"],weights = [0.7,0.1,0.1,0.1],k=1)[0]
    if call_result in ["Disputed","Challenged"]:
       call_result_reason = random.choice(["Trade Difference", "CSA difference", "Valuation difference", "Timing differnce"])
    else:
       call_result_reason = 'NA'

    calls = {

        "Call_ID": f"CALL{i+1:04}",
        "Call_source_system":call_source,
        "Agreement_id": f"CSA_CP{i+1:04}",
        "Margin_type": mar_type,
        "Call_direction":call_dir,
        "Call_amount":mar_amount,
        "Call_result":call_result,
        "Call_result_reason":call_result_reason,
        "Call_date": call_date.strftime("%d-%b-%Y"),


    }
    margin_calls.append(calls)


df_margincalls = pd.DataFrame(margin_calls)

In [38]:
df_margincalls.to_csv("df_margincalls.csv",index=False)